train on the same classes of ABC and ABD and a model on the entire set anf the merg onmiting C and D classes that were not seen

Import Libraries

In [1]:
import pandas as pd
import numpy as np
import stats
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
import statistics 
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, ConvLSTM2D, Dropout,BatchNormalization, Bidirectional, Conv1D, MaxPooling1D, TimeDistributed, Flatten
from tensorflow.keras.utils import plot_model
import graphviz

from scipy import stats
from sklearn.preprocessing import normalize, RobustScaler, OneHotEncoder
from sklearn.model_selection import KFold

User Defined Functions

In [56]:
def load_data_to_memory(path, activities = []):
    # load dataset to memory from a designated path and transform
    # Inputs
    # ----------
    # path: the path were the csv file is stored
    # Outputs
    # ----------
    # frame: the formated dataset in a tabular form as a pandas dataframe
    all_files = glob.glob(path + "/*.txt")

    li = []

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=None)
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    frame.columns = ['User', 'Activity', 'timestamp', 'X', 'Y', 'Z']
    frame.Z.replace(regex=True, inplace=True, to_replace=r';', value=r'')
    frame['Z'] = frame.Z.astype(np.float64)
    frame.dropna(axis=0, how='any', inplace=True)
    
    if len(activities) > 0:
        frame = frame[frame['Activity'].isin(activities)]
        
    print('A dataset of ' + str(frame.shape[0]) + ' rows and ' + str(frame.shape[1]) + ' columns was loaded to memory')
    return(frame)

def export_user_data(data, user, categories):
    # extract certain types of categories from certain users
    # Inputs
    # ----------
    # data: pandas dataframe
    # user: list, the requested users
    # categories: list, the requested categories
    # Outputs
    # ----------
    # dataset: a pandas dataframe of the requested users and categories
    dataset = data[(data['Activity'].isin(categories)) & (data['User'].isin(user))]
    dataset = dataset.drop(['User', 'timestamp'], axis = 1)
    return dataset

def format1D(X, y, window_size, stride, features, encoder):
    X = np.array(X)
    Xs, ys = [], []
    for i in range(0, len(X) - window_size, stride):
        labels = y.iloc[i: i + window_size]
        try:
            modelabel = stats.mode(labels)[0][0]
            v = X[i:(i + window_size)]
            Xs.append(v)
            ys.append(modelabel)
        except: continue
    Xs, ys = np.array(Xs), np.array(ys)
    ys = encoder.transform(ys.reshape(-1, 1))
    return Xs, ys
    
def format_set(X, y, window_size, stride, steps, dimentions, sub_window_size, features, encoder):
    # function to split the dataset to smaller partitions based on a sliding window with overlap
    # Inputs
    # ----------
    # X: pandas dataframe
    # y: list, the requested users
    # window_size: The number of time steps to include in the model input
    # stride: The number of observations between windows
    # steps: the number of subwindows inside of the window
    # dimentions: The number of the dimentions to fit the data using ConvLSTM
    # sub_window_size: The size of the internal window
    # features: The number of predictors
    # encoder: The dummy encoder of the categorical y
    # ----------
    # Xs: a numpy array of the Xs of the model
    # Xy: a numpy array of the ys of the model
    X = np.array(X)
    Xs, ys = [], []
    for i in range(0, len(X) - window_size, stride):
        labels = y.iloc[i: i + window_size]
        try:
            modelabel = stats.mode(labels)[0][0]
            v = X[i:(i + window_size)]
            Xs.append(v)
            ys.append(modelabel)
        except: continue
    Xs, ys = np.array(Xs), np.array(ys)
    
    Xs = Xs.reshape((Xs.shape[0], steps, dimentions, sub_window_size, features))
    ys = encoder.transform(ys.reshape(-1, 1))
    return Xs, ys

def define_conv_lstm_model(n_steps, n_length, n_features, categories_number):
    # Define and compile the ConvLSTM model
    # Inputs
    # ----------
    # n_steps: number of internal windows
    # n_length: length of internal windows
    # n_features: number of predictors
    # categories_number: Number of categories of ys
    # Outputs
    # ----------
    # model: the compiled ConvLSTM model
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(categories_number, activation='softmax'))
    
    model.compile(
      loss='categorical_crossentropy',
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
      metrics=['acc']
    )
    return model
    
def define_1DCNNLSTM_model(n_timesteps, n_features, categories_number):
    # Define and compile the ConvLSTM model
    # Inputs
    # ----------
    # n_steps: number of internal windows
    # n_length: length of internal windows
    # n_features: number of predictors
    # categories_number: Number of categories of ys
    # Outputs
    # ----------
    # model: the compiled ConvLSTM model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(categories_number, activation='softmax'))

    model.compile(
      loss='categorical_crossentropy',
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
      metrics=['acc']
    )
    return model

def get_evaluation_metrics(model, X_test, y_test):
    # Evaluate the model 
    # Inputs
    # ----------
    # model: the compiled ConvLSTM model
    # X_test: The Xs of test/unseen data
    # y_test: The true value of ys
    # Outputs
    # ----------
    # cm: confusion matrix true vs predicted
    # loss: the loss of the fitted model
    # acc: accuracy of the fitted model
    # recalls: true positives
    predictions = model.predict(X_test)
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    y_pred = tf.argmax(predictions, axis=-1)
    y_true = tf.argmax(y_test, axis=-1)
    cm = tf.math.confusion_matrix(tf.reshape(y_true, [-1]),
                             tf.reshape(y_pred, [-1]))
    diagonal = tf.linalg.diag_part(cm)
    recalls = diagonal / tf.reduce_sum(cm, axis=1)
    
    return cm, loss, acc, recalls

def train_conv_lstm_model(model, X_train, y_train, X_test, y_test):
    # Fits the model n the given dataset and produces its scores and history data
    # Inputs
    # ----------
    # model: the compiled ConvLSTM model
    # X_train: The Xs of trained data
    # y_train : The ys of the trained data
    # X_test: The Xs of test data
    # y_test: The true value of ys
    # Outputs
    # ----------
    # model: the fitted model
    # history: histore of the model training
    # cm: confusion matrix true vs predicted
    # loss: the loss of the fitted model
    # acc: accuracy of the fitted model
    # recalls: true positives
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15, restore_best_weights=True)
    
    history = model.fit(
        X_train, y_train,
        epochs=500,
        batch_size=32,
        validation_split=0.3,
        shuffle=True,
        callbacks = es,
        verbose = False
    )
    
    cm, loss, acc, recalls = get_evaluation_metrics(model, X_test, y_test)
    
    return model, history, cm, loss, acc, recalls


def clone_model(origin_model):
    # Created a deep copy of any given model
    # Inputs
    # ----------
    # origin_model: The model intended to be copied
    # Outputs
    # ----------
    # model_copy: The deep copy of the input model
    model_copy= keras.models.clone_model(origin_model)
    model_copy.build() # replace 10 with number of variables in input layer
    
    model_copy.compile(
      loss='categorical_crossentropy',
      optimizer='adam',
      metrics=['acc']
    )
    model_copy.set_weights(origin_model.get_weights())
    return model_copy

def merge_models(models):
    # merges given models using the average of their weights
    # Inputs
    # ----------
    # models: A list of models to be merged
    # Outputs
    # ----------
    # new_model: Model produced by weight averaging of the input models
    temp_weights = models[0].get_weights()
    
    for index, model in enumerate(models):
        if index == 0: continue
        temp_weights = np.add(temp_weights, model.get_weights())
    merged_weights = temp_weights/len(models)
    new_model = clone_model(models[0])
    new_model.set_weights(merged_weights)
    return new_model

def get_high_entropy_preds(model, X, percentile):
    # Given a model returns the indexes of Xs giving the most uncertain predictions of y true based of the probabilities of classes
    # Inputs
    # ----------
    # model: The model we intent to use
    # X: The Xs used to predict and calculate entropy of predictions
    # percentile
    # Outputs
    # ----------
    # indexes: The indexes referring to Xs
    entropies = stats.entropy(model.predict(X), axis = 1)
    entropy_threshold = np.percentile(entropies, percentile)
    indexes = entropies>=entropy_threshold
    return indexes

def fetch_round_batch(X, y, round_batch_size):
    # Returns the indexes of the instances used in each round and the remaining set
    # Inputs
    # ----------
    # X: the Xs instances of the set
    # ys: The y true of the set
    # round_batch_size: The number of observation to pass to the model at each round
    # Outputs
    # ----------
    # X: The remainig Xs of the initial set
    # X_batch: The Xs which are going to be passed through the model
    # y: The remaining ys of the initial set
    # y_batch: The ys which are going to be passed through the model
    indices = np.arange(y.shape[0])
    np.random.shuffle(indices)
    
    X = X[indices]
    y = y[indices]
    
    X_batch = X[0:round_batch_size]
    X = np.delete(X,  np.s_[0:round_batch_size], axis = 0)
    
    y_batch = y[0:round_batch_size]
    y = np.delete(y,  np.s_[0:round_batch_size], axis = 0)
    
    return X, X_batch, y, y_batch

def run_experiment(model, no_of_rounds, combinations, threshold = None, start = 1):
    # Simulates and runs the federated learning process 
    # Inputs
    # ----------
    # X: the Xs instances of the set
    # ys: The y true of the set
    # round_batch_size: The number of observation to pass to the model at each round
    # Outputs
    # ----------
    # X: The remainig Xs of the initial set
    # X_batch: The Xs which are going to be passed through the model
    # y: The remaining ys of the initial set
    # y_batch: The ys which are going to be passed through the model
    start_time = time.time()
    federated_model = clone_model(model)
    subsets = []
    best_model = None
    best_loss = 1000
    best_acc = 0
    best_recalls = None
    best_cm = None
    counter = 0
    
    for combination in combinations:
        subset = export_user_data(df, combination[0], combination[1])
        X, y = format_set(subset[['X', 'Y', 'Z']], subset.Activity,
                                window_size, stride, 
                                steps, dimentions, sub_window_size, features, 
                                encoder) 
        round_batch_size = -(-len(X) // no_of_rounds)
        subsets.append([X, y, round_batch_size]) 
        
    loss_bin = []
    acc_bin = []

    for round in range(no_of_rounds):
        edge_models = []
        for index, subset in enumerate(subsets):
            subsets[index][0], X_train, subsets[index][1], y_train = fetch_round_batch(subset[0], subset[1], subset[2])
            
            if round>=start and threshold is not None:
                indices = get_high_entropy_preds(federated_model, X_train, threshold)
                X_train = X_train[indices]
                y_train = y_train[indices]

            edge_model = clone_model(federated_model)

            edge_model, b_history, b_cm, b_loss, b_acc, b_recalls = train_conv_lstm_model(edge_model, 
                                                                       X_train, y_train, X_test, y_test)

            edge_models.append(edge_model)
        federated_model = merge_models(edge_models)
        cm, loss, acc, recalls = get_evaluation_metrics(federated_model, X_test, y_test)
        loss_bin.append(loss)
        acc_bin.append(acc)
        
        if acc > best_acc:
            best_model = federated_model
            best_loss = loss
            best_acc = acc
            best_recalls = recalls
            best_cm = cm
            best_round = round + 1
            counter = 0

    best_statistics = [no_of_rounds, threshold, best_loss, statistics.stdev(loss_bin), best_acc, statistics.stdev(acc_bin), 
                       best_recalls, time.time() - start_time, best_round]
    return best_statistics 

Solve

In [4]:
df = load_data_to_memory('wisdm-dataset/raw/watch/accel', ['A', 'B', 'C', 'D'])

A dataset of 836612 rows and 6 columns was loaded to memory


In [29]:
window_size = 100 
stride = 20
steps = 4 
dimentions = 1 
sub_window_size = 25 
features = 3
categories = len(set(df.Activity))
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False).fit(df.Activity.values.reshape(-1, 1))

train_users = df.User.unique()[:40]
test_users = df.User.unique()[40:]

test_set = df[df['User'].isin(test_users)]
X_test, y_test = format_set(test_set[['X', 'Y', 'Z']], test_set.Activity,
                            window_size, stride, 
                            steps, dimentions, sub_window_size, features, 
                            encoder)

## Federated Learning
Inthis step I will train various models in subsamples of the entire data containing different instances of said classes in each sample. Next I will merge the models in a single global model.

### Federated Learning by splitting the entire dataset in 2 subsamples containing A,B,C and A,B,D classes respectfully

Full Model

In [33]:
subset_1 = export_user_data(df, train_users[:20], ['A', 'B', 'C'])
subset_2 = export_user_data(df, train_users[20:], ['A', 'B', 'D'])
subset_12 = pd.concat([subset_1,subset_2],ignore_index=True)

In [39]:
full_model = define_conv_lstm_model(steps, sub_window_size, features, categories)

In [40]:
full_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_5 (ConvLSTM2D)  (None, 1, 23, 64)         51712     
_________________________________________________________________
dropout_11 (Dropout)         (None, 1, 23, 64)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 1472)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 32)                47136     
_________________________________________________________________
dropout_12 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 4)                 132       
Total params: 98,980
Trainable params: 98,980
Non-trainable params: 0
__________________________________________________

In [30]:
X_train, y_train = format_set(subset_12[['X', 'Y', 'Z']], subset_12.Activity,
                            window_size, stride, 
                            steps, dimentions, sub_window_size, features, 
                            encoder)

In [41]:
model, history, cm, loss, acc, recalls = train_conv_lstm_model(full_model, X_train, y_train, X_test, y_test)

Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping


In [42]:
acc

0.6985942125320435

In [43]:
loss

0.9988209009170532

In [44]:
recalls

<tf.Tensor: shape=(4,), dtype=float64, numpy=array([0.89154639, 0.99243099, 0.45847039, 0.47468094])>

In [34]:
cm

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[2370,   18,   32,    5],
       [  20, 2217,    9,    0],
       [1015,   37, 1373,    7],
       [ 298,   84,  549, 1498]], dtype=int32)>

1D CNN-LSTM Model

In [14]:
window_size

4

In [57]:
CNNLSTM_model = define_1DCNNLSTM_model(window_size, features, categories)

In [58]:
X_train, y_train = format1D(subset_12[['X', 'Y', 'Z']], subset_12.Activity,
                            window_size, stride, 
                            features, 
                            encoder)

X_test, y_test = format1D(test_set[['X', 'Y', 'Z']], test_set.Activity,
                            window_size, stride, 
                            features, 
                            encoder)

In [59]:
model, history, cm, loss, acc, recalls = train_conv_lstm_model(CNNLSTM_model, X_train, y_train, X_test, y_test)

Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping


In [61]:
recalls

<tf.Tensor: shape=(4,), dtype=float64, numpy=array([0.75876289, 0.99821906, 0.58799342, 0.63153561])>

Federated learning

In [5]:
total_st_no_init = []
for threshold in [None, 50, 70, 90]:
    for no_of_rounds in [2, 4, 10, 14, 20, 30]:
        federated_model = define_conv_lstm_model(steps, sub_window_size, features, categories)
        combinations = [[train_users[:20], ['A', 'B', 'C']],
                           [train_users[:20],['A', 'B', 'D']]]

        st = run_experiment(federated_model, no_of_rounds, combinations, threshold, 1)
        total_st_no_init.append(st)

Restoring model weights from the end of the best epoch.
Epoch 00031: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00019: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00022: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00033: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00022: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00023: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00022: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early s

Restoring model weights from the end of the best epoch.
Epoch 00019: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00056: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00023: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00023: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00029: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00033: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00018: early s

Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00028: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00038: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00028: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00044: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00026: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00039: early s

Restoring model weights from the end of the best epoch.
Epoch 00034: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00019: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00029: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00022: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early s

Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00048: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00019: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00030: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00030: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early s

Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00033: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00027: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00022: early s

Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00028: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00022: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00070: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00023: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early s

Federated learning 1D

In [ ]:
total_st_no_init = []
for threshold in [None, 50, 70, 90]:
    for no_of_rounds in [2, 4, 10, 14, 20, 30]:
        federated_model = define_conv_lstm_model(steps, sub_window_size, features, categories)
        combinations = [[train_users[:20], ['A', 'B', 'C']],
                           [train_users[:20],['A', 'B', 'D']]]

        st = run_experiment(federated_model, no_of_rounds, combinations, threshold, 1)
        total_st_no_init.append(st)

In [6]:
pd.DataFrame(total_st_no_init, columns = ['Number_of_rounds', 'prercentage', 'loss','st.dev_of_loss', 
                                 'accuracy', 'st.dev_of_acc', 'recalls', 'time_to_completion', 'round_of_best_model']).to_csv('best_statistics_no_init_3.csv')

Using an initial subset of A & B classes only

training the full model

In [40]:
subset_init = export_user_data(df, train_users[:10], ['A', 'B'])
subset_1 = export_user_data(df, train_users[10:25], ['A', 'B', 'C'])
subset_2 = export_user_data(df, train_users[25:], ['A', 'B', 'D'])
subset_full = pd.concat([subset_init, subset_1,subset_2],ignore_index=True)

In [41]:
X_train, y_train = format_set(subset_full[['X', 'Y', 'Z']], subset_full.Activity,
                            window_size, stride, 
                            steps, dimentions, sub_window_size, features, 
                            encoder)

In [42]:
full_model = define_conv_lstm_model(steps, sub_window_size, features, categories)
model, history, cm, loss, acc, recalls = train_conv_lstm_model(full_model, X_train, y_train, X_test, y_test)

Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping


In [43]:
loss

1.598001480102539

In [44]:
acc

0.6809693574905396

In [45]:
cm

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[1938,  259,  219,    9],
       [   7, 2238,    1,    0],
       [1149,   65, 1209,    9],
       [ 495,   14,  814, 1106]], dtype=int32)>

In [46]:
recalls

<tf.Tensor: shape=(4,), dtype=float64, numpy=array([0.79917526, 0.99643811, 0.49712171, 0.45533141])>

Federated learning

In [47]:
total_st_init = []
X_train, y_train = format_set(subset_init[['X', 'Y', 'Z']], subset_init.Activity,
                            window_size, stride, 
                            steps, dimentions, sub_window_size, features, 
                            encoder)
starting_model = define_conv_lstm_model(steps, sub_window_size, features, categories)
starting_model, history, cm, loss, acc, recalls = train_conv_lstm_model(starting_model, X_train, y_train, X_test, y_test)

for threshold in [None, 50, 70, 90]:
    for no_of_rounds in [2, 4, 10, 14, 20]:
        federated_model = clone_model(starting_model)
        combinations = [[train_users[10:25], ['A', 'B', 'C']],
                           [train_users[25:],['A', 'B', 'D']]]

        st = run_experiment(federated_model, no_of_rounds, combinations, threshold, 0)
        total_st_init.append(st)

Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00027: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00029: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00040: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00028: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00019: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00031: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00017: early s

Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00022: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00022: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00023: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00035: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00023: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00032: early s

Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00031: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00030: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00026: early s

Restoring model weights from the end of the best epoch.
Epoch 00066: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00043: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00023: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00029: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00030: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00041: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00026: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00023: early s

Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00035: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00026: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00036: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00042: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping


In [48]:
pd.DataFrame(total_st_init, columns = ['Number_of_rounds', 'prercentage', 'loss','st.dev_of_loss', 
                                 'accuracy', 'st.dev_of_acc', 'recalls', 'time_to_completion', 'round_of_best_model']).to_csv('best_statistics_init_2.csv')